# Q2-1

In [173]:
def ngram_probs(filename = 'raw_sentences.txt'):
    bigram_probs = {}
    trigram_probs = {}
    bicount = 0
    tricount = 0
    with open(filename) as file:
        data = file.read()
        lines = data.split('\n')
        for line in lines:
            line = line.split(' ')
            for i in range(len(line) - 1):
                #if line[i].isalpha() and line[i + 1].isalpha():
                bigraph = (line[i].lower(), line[i + 1].lower())
                if bigraph not in bigram_probs:
                    bigram_probs[bigraph] = 1
                else:
                    bigram_probs[bigraph] += 1
                bicount += 1
            for i in range(len(line) - 2):
                #if line[i].isalpha() and line[i + 1].isalpha() and line[i + 2].isalpha():
                trigraph = (line[i].lower(), line[i + 1].lower(), line[i + 2].lower())
                if trigraph not in trigram_probs:
                    trigram_probs[trigraph] = 1
                else:
                    trigram_probs[trigraph] += 1
                tricount += 1
        for k in bigram_probs:
            bigram_probs[k] = round(bigram_probs[k] / bicount, 10)
        for k in trigram_probs:
            trigram_probs[k] = round(trigram_probs[k] / tricount, 10)
    return bigram_probs, trigram_probs

cnt2, cnt3 = ngram_probs()
print(cnt2[('we', 'are')])
# after removing space in calculating

0.0034562629


# Q2-2

In [177]:
import numpy as np
cnt2, cnt3 = ngram_probs()
def prob3(bigram,cnt2 = cnt2,cnt3 = cnt3):        
    p2 = np.log10(cnt2[bigram])
    prob = {}
    for key in cnt3:
        if bigram[0] in key and bigram[1] in key:
            p3 = np.log10(cnt3[key])
            for i in key:
                if i not in bigram:
                    prob[i] = p3 - p2
    return prob

In [178]:
p = prob3(('we','are'))
print(p['family'])

-2.28986864111


In [181]:
def predict_max(starting, cnt2 = cnt2, cnt3 = cnt3):
    res = []
    res.append(starting[0])
    res.append(starting[1])
    while starting[1] and len(res)<=15:
        prob = prob3(starting)
        max_key = max(prob, key=lambda k: prob[k])
        res.append(max_key)
        print(res)
        starting = (starting[1],max_key)
    return res
    #return list_of_words

# Raise a problem when the bigram is not exist (use some smoth method?)

In [182]:
sent = predict_max(('we','are'))
assert sent[-1] == '.' or len(sent) <=15
print(' '.join(sent))


['we', 'are', 'going']
['we', 'are', 'going', 'to']
['we', 'are', 'going', 'to', 'are']


KeyError: ('to', 'are')

In [183]:
def predict_beam(bigram, beam_size = 4, sent_length = 10, cnt2 = cnt2, cnt3 = cnt3):
    list_of_sentence = []
    heap = [bigram]
    sentences = []
    curr_sen = ''
    while heap:
        curr_bigram = heapq.heappop(heap)
        for w in next_list:
            curr_sen = curr_sen + w
            sentences.append(curr_sen)
            if curr_sen[-1] == '.' and len(curr_sen) < 10:
                list_of_sentence.append(curr_sen)
        beam = []
        next_list = []
        d = prob3(curr_bigram)
        for k, v in d.items():
            heapq.heappush(beam, (v, k))
        while len(beam) > beam_size:
            heapq.heappop(beam)
        while beam:
            next_w = heapq.heappop(beam)[1]
            next_list.append(next_w)
    return list_of_sentence
